In [1]:
from src import pysetperm as psp
import numpy as np
import pandas as pd

In [2]:
n_perms = 30000
cores = 6
# +-2kb of gene definition: range_modification=2000
gene_def_plus=2000
# can set minimum size of the candidate gene set.
min_size=10
annotations = psp.AnnotationSet(annotation_file='data/genes.txt', range_modification=gene_def_plus)
function_sets = psp.FunctionSets(function_set_file='data/kegg.txt', min_set_size=min_size, annotation_obj=annotations)

In [3]:
# specific inputs
e_candidates = psp.Variants(variant_file='data/eastern-0.000228-candidate.snps.bed.gz')
e_candidates.annotate_variants(annotation_obj=annotations)
e_background = psp.Variants(variant_file='data/pbsnj-bg.snps.bed.gz')
e_background.annotate_variants(annotation_obj=annotations)

# central can use eastern background.
c_candidates = psp.Variants(variant_file='data/central-0.000192-candidate.snps.bed.gz')
c_candidates.annotate_variants(annotation_obj=annotations)



In [4]:
i_candidates = psp.Variants(variant_file='data/ancestral-0.005-candidate.snps.bed.gz')
i_candidates.annotate_variants(annotation_obj=annotations)
i_background = psp.Variants(variant_file='data/ancestral-bg.bed.gz')
i_background.annotate_variants(annotation_obj=annotations)

In [5]:
i_background.variants

+--------------+-----------+-----------+
| Chromosome   | Start     | End       |
| (category)   | (int32)   | (int32)   |
|--------------+-----------+-----------|
| 1            | 1235069   | 1235070   |
| 1            | 1238849   | 1238850   |
| 1            | 1252699   | 1252700   |
| 1            | 1278319   | 1278320   |
| ...          | ...       | ...       |
| 22           | 48777099  | 48777100  |
| 22           | 48779099  | 48779100  |
| 22           | 48780899  | 48780900  |
| 22           | 48784099  | 48784100  |
+--------------+-----------+-----------+
Unstranded PyRanges object has 295,645 rows and 3 columns from 23 chromosomes.
For printing, the PyRanges was sorted on Chromosome.

In [6]:
# test objects
e_test_obj = psp.TestObject(e_candidates,
                            e_background,
                            function_sets,
                            n_cores=cores)

In [7]:
c_test_obj = psp.TestObject(c_candidates,
                            e_background,
                            function_sets,
                            n_cores=cores)

In [8]:
i_test_obj = psp.TestObject(i_candidates,
                            i_background,
                            function_sets,
                            n_cores=cores)

In [9]:
e_permutations = psp.Permutation(e_test_obj, n_perms, cores)
c_permutations = psp.Permutation(c_test_obj, n_perms, cores)


In [10]:
i_permutations = psp.Permutation(i_test_obj, n_perms, cores)

In [11]:
# distributions across permutations
e_per_set = psp.SetPerPerm(e_permutations,
                           function_sets,
                           e_test_obj,
                           cores)

c_per_set = psp.SetPerPerm(c_permutations,
                           function_sets,
                           c_test_obj,
                           cores)

i_per_set = psp.SetPerPerm(i_permutations,
                           function_sets,
                           i_test_obj,
                           cores)

In [12]:
# results tables
def make_results_table(test_obj, function_set_obj, set_perm_obj):
    out = function_set_obj.function_sets.groupby('Id', as_index=False).agg({'FunctionName': pd.Series.unique})
    out = out[out['Id'].isin(function_set_obj.function_array2d_ids)]
    out['n_candidates'] = test_obj.n_candidate_per_function
    out['mean_n_resample'] = set_perm_obj.mean_per_set
    out['emp_p_e'] = set_perm_obj.p_enrichment
    out['emp_p_d'] = set_perm_obj.p_depletion
    out['fdr_e'] = psp.fdr_from_p_matrix(set_perm_obj.set_n_per_perm, out['emp_p_e'], method='enrichment')
    out['fdr_d'] = psp.fdr_from_p_matrix(set_perm_obj.set_n_per_perm, out['emp_p_d'], method='depletion')
    out['BH_fdr_e'] = psp.p_adjust_bh(out['emp_p_e'])
    out['BH_fdr_d'] = psp.p_adjust_bh(out['emp_p_d'])
    out = out.sort_values('emp_p_e')
    return out


In [13]:
e_results = make_results_table(e_test_obj, function_sets, e_per_set)
c_results = make_results_table(c_test_obj, function_sets, c_per_set)
i_results = make_results_table(i_test_obj, function_sets, i_per_set)

In [14]:
e_results.sort_values('fdr_e')

,Id,FunctionName,n_candidates,mean_n_resample,emp_p_e,emp_p_d,fdr_e,fdr_d,BH_fdr_e,BH_fdr_d
226,hsa04658,Th1 and Th2 cell differentiation,10,3.414433,0.001333,0.999767,0.248200,1.0,0.386987,0.999767
334,hsa05169,Epstein-Barr virus infection,14,6.111533,0.002733,0.999000,0.256233,1.0,0.386987,0.999767
4,hsa00051,Fructose and mannose metabolism,5,1.321300,0.003933,0.999300,0.263433,1.0,0.386987,0.999767
47,hsa00520,Amino sugar and nucleotide sugar metabolism,6,1.749700,0.004300,0.999300,0.263433,1.0,0.386987,0.999767
192,hsa04260,Cardiac muscle contraction,11,5.433900,0.006000,0.998067,0.263433,1.0,0.431986,0.999767
...,...,...,...,...,...,...,...,...,...,...
348,hsa05216,Thyroid cancer,0,1.657367,1.000000,0.171394,1.000000,1.0,1.000000,0.999767
139,hsa03460,Fanconi anemia pathway,0,2.085067,1.000000,0.113196,1.000000,1.0,1.000000,0.999767
118,hsa03020,RNA polymerase,0,0.631633,1.000000,0.521249,1.000000,1.0,1.000000,0.999767
17,hsa00220,Arginine biosynthesis,0,0.597633,1.000000,0.539515,1.000000,1.0,1.000000,0.999767


In [15]:
c_results.sort_values('fdr_e')

,Id,FunctionName,n_candidates,mean_n_resample,emp_p_e,emp_p_d,fdr_e,fdr_d,BH_fdr_e,BH_fdr_d
153,hsa04060,Cytokine-cytokine receptor interaction,20,6.010667,0.000033,1.000000,0.000000,1.0,0.006000,1.0
369,hsa05340,Primary immunodeficiency,6,0.681567,0.000033,1.000000,0.000000,1.0,0.006000,1.0
317,hsa05140,Leishmaniasis,8,1.877033,0.000333,1.000000,0.017822,1.0,0.039999,1.0
154,hsa04061,Viral protein interaction with cytokine and cy...,6,1.381167,0.002500,0.999567,0.115817,1.0,0.224993,1.0
150,hsa04050,Cytokine receptors,9,3.270133,0.004333,0.998833,0.178587,1.0,0.297990,1.0
...,...,...,...,...,...,...,...,...,...,...
94,hsa01006,Prenyltransferases,0,0.681333,1.000000,0.481284,1.000000,1.0,1.000000,1.0
110,hsa03010,Ribosome,0,1.126500,1.000000,0.321989,1.000000,1.0,1.000000,1.0
280,hsa04940,Type I diabetes mellitus,0,0.991100,1.000000,0.354988,1.000000,1.0,1.000000,1.0
11,hsa00100,Steroid biosynthesis,0,0.400467,1.000000,0.664111,1.000000,1.0,1.000000,1.0


In [16]:
i_results.sort_values('fdr_e')

,Id,FunctionName,n_candidates,mean_n_resample,emp_p_e,emp_p_d,fdr_e,fdr_d,BH_fdr_e,BH_fdr_d
124,hsa03036,Chromosome and associated proteins,83,54.463833,0.000133,0.999933,0.017633,1.000000,0.047998,0.999933
26,hsa00310,Lysine degradation,9,2.839233,0.001000,0.999833,0.090583,1.000000,0.179994,0.999933
119,hsa03021,Transcription machinery,19,9.437100,0.001733,0.999333,0.109322,1.000000,0.200721,0.999933
139,hsa03460,Fanconi anemia pathway,7,2.004300,0.002367,0.999567,0.110458,1.000000,0.200721,0.999933
351,hsa05219,Bladder cancer,6,1.618400,0.003500,0.999433,0.137627,1.000000,0.200721,0.999933
...,...,...,...,...,...,...,...,...,...,...
201,hsa04371,Apelin signaling pathway,7,10.033033,0.908736,0.177461,1.000000,0.481385,1.000000,0.967968
306,hsa05033,Nicotine addiction,2,3.808700,0.916603,0.237359,1.000000,0.591793,1.000000,0.999933
241,hsa04723,Retrograde endocannabinoid signaling,7,10.407100,0.922803,0.149728,1.000000,0.406362,1.000000,0.855591
173,hsa04136,Autophagy - other,1,1.902800,0.882404,0.405886,1.000000,0.865337,1.000000,0.999933


In [17]:
# join objects
# test objs
ce_test_obj = psp.TestObject.add_objects(c_test_obj,e_test_obj)
ci_test_obj = psp.TestObject.add_objects(c_test_obj,i_test_obj)
ei_test_obj = psp.TestObject.add_objects(e_test_obj,i_test_obj)
cei_test_obj = psp.TestObject.add_objects(ce_test_obj,i_test_obj)

# n per permuation objs
ce_per_set=psp.SetPerPerm.join_objects(c_per_set,e_per_set)
ci_per_set=psp.SetPerPerm.join_objects(c_per_set,i_per_set)
ei_per_set=psp.SetPerPerm.join_objects(e_per_set,i_per_set)
cei_per_set=psp.SetPerPerm.join_objects(ce_per_set,i_per_set)

In [18]:
# joint results
ce_results = make_results_table(ce_test_obj, function_sets, ce_per_set)
ci_results = make_results_table(ci_test_obj, function_sets, ci_per_set)
ei_results = make_results_table(ei_test_obj, function_sets, ei_per_set)
cei_results = make_results_table(cei_test_obj, function_sets, cei_per_set)

In [19]:
ce_results.sort_values('fdr_e')

,Id,FunctionName,n_candidates,mean_n_resample,emp_p_e,emp_p_d,fdr_e,fdr_d,BH_fdr_e,BH_fdr_d
153,hsa04060,Cytokine-cytokine receptor interaction,27,11.948967,0.000067,1.000000,0.003517,1.000000,0.0120,1.0
317,hsa05140,Leishmaniasis,13,3.730400,0.000067,1.000000,0.003517,1.000000,0.0120,1.0
369,hsa05340,Primary immunodeficiency,8,1.362767,0.000100,1.000000,0.004544,1.000000,0.0120,1.0
226,hsa04658,Th1 and Th2 cell differentiation,18,6.884133,0.000200,0.999967,0.006887,1.000000,0.0144,1.0
4,hsa00051,Fructose and mannose metabolism,10,2.672700,0.000200,0.999967,0.006887,1.000000,0.0144,1.0
...,...,...,...,...,...,...,...,...,...,...
17,hsa00220,Arginine biosynthesis,0,1.210167,1.000000,0.285057,1.000000,1.000000,1.0000,1.0
73,hsa00730,Thiamine metabolism,0,2.156133,1.000000,0.087997,1.000000,0.870602,1.0000,1.0
6,hsa00053,Ascorbate and aldarate metabolism,0,0.683733,1.000000,0.500350,1.000000,1.000000,1.0000,1.0
81,hsa00830,Retinol metabolism,0,2.369767,1.000000,0.087230,1.000000,0.870602,1.0000,1.0


In [20]:
ci_results.sort_values('fdr_e')

,Id,FunctionName,n_candidates,mean_n_resample,emp_p_e,emp_p_d,fdr_e,fdr_d,BH_fdr_e,BH_fdr_d
369,hsa05340,Primary immunodeficiency,8,1.382700,0.000100,1.000000,0.014233,1.000000,0.035999,1.0
317,hsa05140,Leishmaniasis,12,3.798033,0.000267,0.999967,0.023817,1.000000,0.047998,1.0
153,hsa04060,Cytokine-cytokine receptor interaction,25,12.245500,0.000500,0.999800,0.032078,1.000000,0.059998,1.0
217,hsa04620,Toll-like receptor signaling pathway,13,5.135400,0.001500,0.999700,0.080333,1.000000,0.124796,1.0
334,hsa05169,Epstein-Barr virus infection,23,12.023267,0.001733,0.999367,0.080333,1.000000,0.124796,1.0
...,...,...,...,...,...,...,...,...,...,...
152,hsa04054,Pattern recognition receptors,1,3.105167,0.956868,0.175594,1.000000,0.853864,1.000000,1.0
283,hsa04961,Endocrine and other factor-regulated calcium r...,5,8.691867,0.957001,0.100730,1.000000,0.694341,1.000000,1.0
162,hsa04080,Neuroactive ligand-receptor interaction,24,32.069167,0.961935,0.060565,1.000000,0.603028,1.000000,1.0
284,hsa04962,Vasopressin-regulated water reabsorption,2,4.052800,0.933336,0.203027,1.000000,0.916180,1.000000,1.0


In [21]:
ei_results.sort_values('fdr_e')

,Id,FunctionName,n_candidates,mean_n_resample,emp_p_e,emp_p_d,fdr_e,fdr_d,BH_fdr_e,BH_fdr_d
124,hsa03036,Chromosome and associated proteins,148,110.681300,0.000133,0.999967,0.019733,1.000000,0.044999,0.999967
26,hsa00310,Lysine degradation,15,5.754800,0.000433,0.999900,0.027867,1.000000,0.044999,0.999967
226,hsa04658,Th1 and Th2 cell differentiation,17,6.762467,0.000433,0.999800,0.027867,1.000000,0.044999,0.999967
334,hsa05169,Epstein-Barr virus infection,25,11.916933,0.000500,0.999767,0.027867,1.000000,0.044999,0.999967
327,hsa05162,Measles,17,7.649533,0.001000,0.999667,0.040187,1.000000,0.071998,0.999967
...,...,...,...,...,...,...,...,...,...,...
290,hsa04973,Carbohydrate digestion and absorption,4,5.440133,0.818406,0.345488,1.000000,0.927661,1.000000,0.999967
136,hsa03430,Mismatch repair,1,1.694500,0.829806,0.480684,1.000000,1.000000,1.000000,0.999967
63,hsa00592,alpha-Linolenic acid metabolism,1,1.692367,0.830372,0.488317,1.000000,1.000000,1.000000,0.999967
216,hsa04614,Renin-angiotensin system,1,1.721500,0.835372,0.476151,1.000000,1.000000,1.000000,0.999967


In [22]:
cei_results.sort_values('fdr_e')

,Id,FunctionName,n_candidates,mean_n_resample,emp_p_e,emp_p_d,fdr_e,fdr_d,BH_fdr_e,BH_fdr_d
226,hsa04658,Th1 and Th2 cell differentiation,25,10.232167,0.000067,1.000000,0.002300,1.000000,0.008000,1.000000
369,hsa05340,Primary immunodeficiency,10,2.063900,0.000067,1.000000,0.002300,1.000000,0.008000,1.000000
317,hsa05140,Leishmaniasis,17,5.651400,0.000067,1.000000,0.002300,1.000000,0.008000,1.000000
334,hsa05169,Epstein-Barr virus infection,37,18.134800,0.000100,0.999933,0.003808,1.000000,0.009000,1.000000
361,hsa05235,PD-L1 expression and PD-1 checkpoint pathway i...,25,12.944600,0.000567,0.999800,0.020056,1.000000,0.030856,1.000000
...,...,...,...,...,...,...,...,...,...,...
1,hsa00020,Citrate cycle (TCA cycle),1,2.298733,0.905136,0.323089,1.000000,1.000000,1.000000,1.000000
143,hsa04015,Rap1 signaling pathway,41,48.358133,0.905136,0.127562,1.000000,0.748539,1.000000,0.969718
54,hsa00536,Glycosaminoglycan binding proteins,28,34.099933,0.901403,0.138695,1.000000,0.748539,1.000000,0.977175
322,hsa05146,Amoebiasis,14,20.375600,0.962101,0.067364,1.000000,0.748539,1.000000,0.757850
